# Exploratory Notebook

The purpose of this notebook is to explore the data, implement any processing steps and output cleaned dataset for further analysis or learning.

## Assignment

- What insights can you provide the business based on the data?
- Please describe the process and tools that you utilized to ‘clean’ and analyze the data.
- Are there customer variables that are closely related to renewals or churn? 
- How would you communicate your findings to a business stakeholder responsible for driving retention?
- How would you visualize this data in a concise, easily consumable format?  Feel free to provide example view.
- Would you make a recommendation to the business based on this info?
- What additional information would you look for in order to expand your analysis?
- Any other items you would want to add?

## Notes

### Provided Data

    'userid': unique id
    'first_purchase_date': 
    'renewal_date': str,
    'renewal_amount_due_usd': float,
    'licenses': num licenses,
    'assigned_licenses': int,
    'market': {'ENG EMEA', 'NON ENG EMEA', 'ROW', nan},
    'billing_country': country code str,
    'billing_state': city free text str,
    'channel': offline/online
    'total_meetings_30days': int,
    'total_meetings_30to60days': int,
    'meetingdays': int,
    'total_participants': int,
    'webcam_duration': float,
    'audio_duration': float,
    'Renew_Status': {CHURN, RENEW}
  

In [2]:
import pandas as pd
import numpy as np

In [2]:
dtypes = {
    'userid': str,
    'first_purchase_date': str,
    'renewal_date': str,
    'renewal_amount_due_usd': float,
    'licenses': int,
    'assigned_licenses': int,
    'market': str,
    'billing_country': str,
    'billing_state': str,
    'channel': str,
    'total_meetings_30days': int,
    'total_meetings_30to60days': int,
    'meetingdays': int,
    'total_participants': int,
    'webcam_duration': float,
    'audio_duration': float,
    'Renew_Status': str
}

In [2]:
df = pd.read_csv('../data/data.csv')
print("Total rows: {}".format(len(df)))

Total rows: 57599


In [42]:
df1 = pd.read_csv('../data/data.csv')


In [43]:
df1.count()


userid                       57599
first_purchase_date          57597
renewal_date                 57599
renewal_amount_due_usd       57599
licenses                     57599
assigned_licenses            57599
market                       13445
billing_country              53839
billing_state                45518
channel                      57599
total_meetings_30days        57599
total_meetings_30to60days    57599
meetingdays                  57599
total_participants           40526
webcam_duration              40351
audio_duration               57599
Renew_Status                 57599
dtype: int64

## Inspect Fields

- inspect existing fields
- add extra useful data fields 

In [3]:
df.count()

userid                       57599
first_purchase_date          57597
renewal_date                 57599
renewal_amount_due_usd       57599
licenses                     57599
assigned_licenses            57599
market                       13445
billing_country              53839
billing_state                45518
channel                      57599
total_meetings_30days        57599
total_meetings_30to60days    57599
meetingdays                  57599
total_participants           40526
webcam_duration              40351
audio_duration               57599
Renew_Status                 57599
dtype: int64

### `userid`

- no missing data
- NOT unique id per subscription
- unusal id formatting; length of id changes and recorded in scientific notation??

In [78]:
ids[57595:]

[9.22987e+17, 9.37721e+17, 9.81945e+17, 9.9101e+17]

#### Not hugely important, just convert to ints and investigate later...

In [5]:
df.userid = df.userid.apply(lambda x: int(x))

In [6]:
print("Total subscriptions: {}".format(len(set(df.userid))))

Total subscriptions: 56559


### `first_purchase_date`

- 2 rows missing field

In [7]:
df.loc[df.first_purchase_date.isnull()]

,userid,first_purchase_date,renewal_date,renewal_amount_due_usd,licenses,assigned_licenses,market,billing_country,billing_state,channel,total_meetings_30days,total_meetings_30to60days,meetingdays,total_participants,webcam_duration,audio_duration,Renew_Status
19709,798576000000000000,NaN,6/1/2018,6720,10,0,NaN,US,NV,offline,8,2,7,12.0,0.0,644.0,RENEW
35264,8695830000000000000,NaN,6/15/2018,5400,10,0,NaN,US,MI,offline,1,2,1,2.0,0.0,8.0,RENEW


In [8]:
df.first_purchase_date = pd.to_datetime(df.first_purchase_date, infer_datetime_format=True)

In [9]:
min(list(df.first_purchase_date))

Timestamp('2010-01-22 00:00:00')

In [10]:
max(list(df.first_purchase_date))

Timestamp('2018-01-15 00:00:00')

### `first_purchase_year`

In [123]:
df['first_purchase_year'] = df.first_purchase_date.apply(lambda x: x.year)

### `first_purchase_month`

In [12]:
df['first_purchase_month'] = df.first_purchase_date.apply(lambda x: x.month)

### `renewal_date`

- date in 2018 where subscription fee is due if subscriber is renewing

In [13]:
df.renewal_date = pd.to_datetime(df.renewal_date, infer_datetime_format=True)

In [14]:
min(list(df.renewal_date))

Timestamp('2018-02-13 00:00:00')

In [15]:
max(list(df.renewal_date))

Timestamp('2018-06-30 00:00:00')

In [16]:
df['renewal_month'] = df.renewal_date.apply(lambda x: x.month)

In [17]:
df['renewal_year'] = df.renewal_date.apply(lambda x: x.year)

In [18]:
set(df.renewal_year)

{2018}

### `years_active`

- calculate how many years subscriber has been active
- maybe can make this months later but years is fine for now

In [19]:
def _get_years_active(row):
    try:
        y = row.renewal_year - row.first_purchase_year
        return int(y)
    except:
        return np.nan

In [20]:
df['years_active'] = df.apply(_get_years_active, axis=1)

In [28]:
l = list(df.dropna(subset=['years_active']).years_active)

print("Min: {}".format(min(l)))
print("Max: {}".format(max(l)))
print("Mean: {}".format(np.mean(l)))

Min: 0.0
Max: 8.0
Mean: 3.3717381113599667


### `renewal_amount_due_usd`

In [52]:
df.renewal_amount_due_usd = df.renewal_amount_due_usd.apply(lambda x: float(x))

In [114]:
r = list(df.renewal_amount_due_usd)

print("Min amount = ${}".format(min(r)))
print("Max amount = ${}".format(max(r)))
print("Mean amount = ${}".format(np.mean(r)))

Min amount = $26.0
Max amount = $849264.0
Mean amount = $1607.585096963489


### `licenses`

* number of licenses per subscription
* current or just total sub licences of all time? 

In [32]:
l = list(df.licenses)

print("Min: {}".format(min(l)))
print("Max: {}".format(max(l)))
print("Mean: {}".format(np.mean(l)))

Min: 1
Max: 27220
Mean: 5.624299032969322


### `assigned_licenses`

- licenses assigned to people
- user can have purchased a license and just not have assigned it to anyone

In [33]:
l = list(df.assigned_licenses)

print("Min: {}".format(min(l)))
print("Max: {}".format(max(l)))
print("Mean: {}".format(np.mean(l)))

Min: 0
Max: 25154
Mean: 5.590062327470963


In [44]:
a = df.loc[df.assigned_licenses > df.licenses]
b = df.loc[df.assigned_licenses < df.licenses]
c = df.loc[df.assigned_licenses == df.licenses]

print("assigned == licensed: {}".format(len(c)))
print("assigned > licensed: {}".format(len(a)))
print("assigned < licensed: {}".format(len(b)))

assigned == licensed: 49787
assigned > licensed: 714
assigned < licensed: 7098


### `renewal_amount_due_usd_per_license`

In [54]:
df['renewal_amount_due_usd_per_license'] = df.apply(lambda r: r.renewal_amount_due_usd / r.licenses, axis=1)

In [59]:
l = list(df.renewal_amount_due_usd_per_license)

print("Min: ${}".format(min(l)))
print("Max: ${}".format(max(l)))
print("Mean: ${}".format(np.mean(l)))

Min: $10.666666666666666
Max: $1813.0
Mean: $378.2128609871642


### `market`

* ENG EMEA
* NON ENG EMEA
* ROW (Rest of World)
* **Lots** of missing fields which can be inferred using billing country/state

In [66]:
eng = df.loc[df.market == 'ENG EMEA']
non_eng = df.loc[df.market == 'NON ENG EMEA']
row = df.loc[df.market == 'ROW']
other = df.loc[df.market.isnull()]
total = len(df)

print("Total ENG: {} [{:.3f}%]".format(len(eng), (len(eng) / total * 100.0)))
print("Total NON ENG: {} [{:.3f}%]".format(len(non_eng), (len(non_eng) / total * 100.0)))
print("Total ROW: {} [{:.3f}%]".format(len(row), (len(row) / total * 100.0)))
print("Total OTHER: {} [{:.3f}%]".format(len(other), (len(other) / total * 100.0)))

Total ENG: 9653 [16.759%]
Total NON ENG: 874 [1.517%]
Total ROW: 2918 [5.066%]
Total OTHER: 44154 [76.658%]


#### Investigate countries

In [82]:
eng_countries = set(eng.dropna(subset=['billing_country']).billing_country)
non_eng_countries = set(non_eng.dropna(subset=['billing_country']).billing_country)
row_countries = set(row.dropna(subset=['billing_country']).billing_country)
other_countries = set(other.dropna(subset=['billing_country']).billing_country)

print("Total ENG countries: {}".format(len(eng_countries)))
print("Total NON ENG countries: {}".format(len(non_eng_countries)))
print("Total ROW countries: {}".format(len(row_countries)))
print("Total OTHER countries: {}".format(len(other_countries)))

Total ENG countries: 68
Total NON ENG countries: 30
Total ROW countries: 28
Total OTHER countries: 6


#### Check overlap...

In [84]:
set.intersection(eng_countries, non_eng_countries, row_countries)

set()

In [98]:
set.intersection(eng_countries, other_countries)

set()

In [97]:
set.intersection(non_eng_countries, other_countries)

{'US'}

In [95]:
set.intersection(row_countries, other_countries)

set()

In [100]:
other_countries

{'BM', 'BS', 'CA', 'KY', 'US', 'VG'}

### Manually fix US 

In [99]:
df.loc[(df.billing_country == 'US') & (df.market == 'NON ENG EMEA')]

,userid,first_purchase_date,renewal_date,renewal_amount_due_usd,licenses,assigned_licenses,market,billing_country,billing_state,channel,...,total_participants,webcam_duration,audio_duration,Renew_Status,first_purchase_year,first_purchase_month,renewal_month,renewal_year,years_active,renewal_amount_due_usd_per_license
3273,197546000000000000,2014-10-30,2018-06-22,258.0,1,1,NON ENG EMEA,US,FL,online,...,46.0,277.05,5523.89,CHURN,2014.0,10.0,6,2018,4.0,258.0
5634,755685,2013-04-28,2018-06-08,468.0,1,1,NON ENG EMEA,US,TX,online,...,74.0,288.00,157.00,RENEW,2013.0,4.0,6,2018,5.0,468.0


In [101]:
df.loc[df.billing_country == 'US', 'market'] = 'ROW'

### Manually fix 'other' countries

In [102]:
other_countries

{'BM', 'BS', 'CA', 'KY', 'US', 'VG'}

#### Bermuda

In [103]:
df.loc[df.billing_country == 'BM', 'market'] = 'ROW'

#### Bahamas

In [104]:
df.loc[df.billing_country == 'BS', 'market'] = 'ROW'

#### Canada

In [105]:
df.loc[df.billing_country == 'CA', 'market'] = 'ROW'

#### Cayman Islands

In [106]:
df.loc[df.billing_country == 'KY', 'market'] = 'ROW'

#### Virgin Islands

In [107]:
df.loc[df.billing_country == 'VG', 'market'] = 'ROW'

### Cleaned `market`

- reduced 'other' significantly
- ~3k rows with missing markets have significanly lower number of subscriptions and hence monetary impact

In [108]:
eng = df.loc[df.market == 'ENG EMEA']
non_eng = df.loc[df.market == 'NON ENG EMEA']
row = df.loc[df.market == 'ROW']
other = df.loc[df.market.isnull()]
total = len(df)

print("Total ENG EMEA: {} [{:.3f}%]".format(len(eng), (len(eng) / total * 100.0)))
print("Total NON ENG EMEA: {} [{:.3f}%]".format(len(non_eng), (len(non_eng) / total * 100.0)))
print("Total ROW: {} [{:.3f}%]".format(len(row), (len(row) / total * 100.0)))
print("Total OTHER: {} [{:.3f}%]".format(len(other), (len(other) / total * 100.0)))

Total ENG EMEA: 9653 [16.759%]
Total NON ENG EMEA: 872 [1.514%]
Total ROW: 44199 [76.736%]
Total OTHER: 2875 [4.991%]


In [110]:
other.loc[(~other.billing_country.isnull())]

,userid,first_purchase_date,renewal_date,renewal_amount_due_usd,licenses,assigned_licenses,market,billing_country,billing_state,channel,...,total_participants,webcam_duration,audio_duration,Renew_Status,first_purchase_year,first_purchase_month,renewal_month,renewal_year,years_active,renewal_amount_due_usd_per_license


In [116]:
r = list(other.renewal_amount_due_usd)

print("Min amount = ${}".format(min(r)))
print("Max amount = ${}".format(max(r)))
print("Mean amount = ${}".format(np.mean(r)))

Min amount = $46.0
Max amount = $3744.0
Mean amount = $349.44347826086954


In [118]:
l = list(other.licenses)

print("Min: {}".format(min(l)))
print("Max: {}".format(max(l)))
print("Mean: {}".format(np.mean(l)))

Min: 1
Max: 8
Mean: 1.0747826086956522


In [119]:
other.loc[other.licenses == 8]

,userid,first_purchase_date,renewal_date,renewal_amount_due_usd,licenses,assigned_licenses,market,billing_country,billing_state,channel,...,total_participants,webcam_duration,audio_duration,Renew_Status,first_purchase_year,first_purchase_month,renewal_month,renewal_year,years_active,renewal_amount_due_usd_per_license
12774,655089,2013-08-24,2018-06-23,3744.0,8,8,NaN,NaN,NaN,online,...,14.0,0.0,1192.95,RENEW,2013.0,8.0,6,2018,5.0,468.0
46705,2882990000000000000,2017-04-18,2018-04-19,2784.0,8,5,NaN,NaN,NaN,online,...,NaN,NaN,0.00,CHURN,2017.0,4.0,4,2018,1.0,348.0


### `billing_country`

- country codes
- missing values


In [128]:
countries = set(list(df.billing_country))
print("Total countries: {}".format(len(countries)))

Total countries: 132


### `billing_state`

- states/cities
- free text 💩
- missing values

In [127]:
states = set(list(df.billing_state))
print("Total states: {}".format(len(states)))

Total states: 1108


### `channel`

- online/offline
- no missing data

In [129]:
online = df.loc[df.channel == 'online']
offline = df.loc[df.channel == 'offline']
total = len(df)

print("Total Online: {} [{:.3f}%]".format(len(online), (len(online) / total * 100.0)))
print("Total Offline: {} [{:.3f}%]".format(len(offline), (len(offline) / total * 100.0)))

Total Online: 40874 [70.963%]
Total Offline: 16725 [29.037%]


### `total_meetings_30days`

- num meetings in last 30 days?

In [131]:
l = list(df.total_meetings_30days)

print("Min: {}".format(min(l)))
print("Max: {}".format(max(l)))
print("Mean: {}".format(np.mean(l)))

Min: 0
Max: 119898
Mean: 35.76174933592597


### `total_meetings_30to60days`

- num meetings in previous 30 days

In [135]:
l = list(df.total_meetings_30to60days)

print("Min: {}".format(min(l)))
print("Max: {}".format(max(l)))
print("Mean: {}".format(np.mean(l)))

Min: 0
Max: 92294
Mean: 20.057935033594333


### `meetingdays`

In [137]:
l = list(df.meetingdays)

print("Min: {}".format(min(l)))
print("Max: {}".format(max(l)))

Min: 0
Max: 31


In [140]:
df.groupby('meetingdays').userid.nunique()

meetingdays
0     16910
1      5245
2      3840
3      3126
4      2690
5      2476
6      2080
7      1753
8      1691
9      1495
10     1317
11     1248
12     1134
13     1038
14      976
15      925
16      876
17      826
18      832
19      812
20      830
21     1063
22     1762
23      889
24      502
25      313
26      233
27      151
28       65
29       44
30       47
31       72
Name: userid, dtype: int64

### `total_participants`

- missing data
- all missing data is missing webcam_duration and audio duration too

In [148]:
l = list(df.dropna(subset=['total_participants']).total_participants)

print("Min: {}".format(min(l)))
print("Max: {}".format(max(l)))
print("Mean: {}".format(np.mean(l)))

Min: 0.0
Max: 83192.0
Mean: 41.027809307605


In [31]:
df.loc[df.total_participants.isnull() & ~df.webcam_duration.isnull()]

,userid,first_purchase_date,renewal_date,renewal_amount_due_usd,licenses,assigned_licenses,market,billing_country,billing_state,channel,...,total_participants,webcam_duration,audio_duration,Renew_Status,first_purchase_year,first_purchase_month,renewal_month,renewal_year,years_active,renewal_amount_due_usd_per_license


In [32]:
df.loc[df.total_participants.isnull() & df.audio_duration != 0.0]

,userid,first_purchase_date,renewal_date,renewal_amount_due_usd,licenses,assigned_licenses,market,billing_country,billing_state,channel,...,total_participants,webcam_duration,audio_duration,Renew_Status,first_purchase_year,first_purchase_month,renewal_month,renewal_year,years_active,renewal_amount_due_usd_per_license


### `webcam_duration`

- missing data
- all rows with missing webcam have 0.0 audio usage

In [149]:
l = list(df.dropna(subset=['webcam_duration']).webcam_duration)

print("Min: {}".format(min(l)))
print("Max: {}".format(max(l)))
print("Mean: {}".format(np.mean(l)))

Min: 0.0
Max: 140688.47
Mean: 218.77620579415628


In [5]:
df.loc[df.webcam_duration.isnull() & df.audio_duration != 0.0]

,userid,first_purchase_date,renewal_date,renewal_amount_due_usd,licenses,assigned_licenses,market,billing_country,billing_state,channel,...,total_participants,webcam_duration,audio_duration,Renew_Status,first_purchase_year,first_purchase_month,renewal_month,renewal_year,years_active,renewal_amount_due_usd_per_license


### Is it ok to just fill na with 0.0?

In [7]:
missing = df.loc[df.webcam_duration.isnull()]

In [12]:
len(missing.loc[missing.Renew_Status == 'CHURN']) / len(missing)

0.350765306122449

In [11]:
len(missing.loc[missing.Renew_Status == 'RENEW']) / len(missing)

0.6492346938775511

### Conclusion = yes


In [35]:
df[['total_participants', 'webcam_duration']] = df[['total_participants', 'webcam_duration']].fillna(value=0.0)

### `audio_duration`

In [26]:
zero = df.loc[df.audio_duration == 0.0]

In [27]:
len(zero.loc[zero.Renew_Status == 'CHURN']) / len(zero)

0.33227328139078816

In [28]:
len(zero.loc[zero.Renew_Status == 'RENEW']) / len(zero)

0.6677267186092118

In [151]:
l = list(df.audio_duration)

print("Min: {}".format(min(l)))
print("Max: {}".format(max(l)))
print("Mean: {}".format(np.mean(l)))

Min: 0.0
Max: 6495908.62
Mean: 2433.7752794319345


### `total_usage_duration`

- combine webcam and audio usage

In [37]:
df['total_usage_duration'] = df.apply(lambda row: row['audio_duration'] + row['webcam_duration'], axis=1)

### `Renew_Status`

* RENEW
* CHURN

In [152]:
renew = df.loc[df.Renew_Status == 'RENEW']
churn = df.loc[df.Renew_Status == 'CHURN']

total = len(df)

print("Total RENEW: {} [{:.3f}%]".format(len(renew), (len(renew) / total * 100.0)))
print("Total CHURN: {} [{:.3f}%]".format(len(churn), (len(churn) / total * 100.0)))

Total RENEW: 46718 [81.109%]
Total CHURN: 10881 [18.891%]


## Output to file

In [41]:
df.to_csv('../data/cleaned_data.csv', index=False)